In [1]:
import os
import time
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from tqdm import tqdm

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
# Настройки
YANDEX_IMAGES_URL = "https://yandex.ru/images/"
SAVE_DIR = "dataset"
CATEGORIES = ["polar bear", "brown bear"]
IMAGES_PER_CATEGORY = 1000

In [3]:
# Создание папок
def create_folders():
    for category in CATEGORIES:
        path = os.path.join(SAVE_DIR, category.replace(" ", "_"))
        os.makedirs(path, exist_ok=True)

In [6]:
# # Запуск Selenium
# def init_driver():
#     options = webdriver.ChromeOptions()
#     options.add_argument("--headless")  # Запуск без UI (ускоряет выполнение)
#     driver = webdriver.Chrome(options=options)
#     return driver

In [6]:
 # Поиск изображений в Yandex
def fetch_image_urls(driver, query, max_images):
    driver.get(YANDEX_IMAGES_URL)
    search_box = driver.find_element(By.NAME, "text")
    search_box.send_keys(query)
    search_box.send_keys(Keys.RETURN)
    time.sleep(3)

    image_urls = set()
    scroll_pause_time = 2
    last_height = driver.execute_script("return document.body.scrollHeight")

    while len(image_urls) < max_images:
        soup = BeautifulSoup(driver.page_source, "html.parser")
        thumbnails = soup.find_all("img", class_="serp-item__thumb")
        
        for img in thumbnails:
            if "src" in img.attrs:
                image_urls.add(img["src"])
            if len(image_urls) >= max_images:
                break
        
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(scroll_pause_time)
        new_height = driver.execute_script("return document.body.scrollHeight")
        
        if new_height == last_height:
            break
        last_height = new_height

    return list(image_urls)

In [8]:
# Загрузка изображений
def download_images(image_urls, category):
    save_path = os.path.join(SAVE_DIR, category.replace(" ", "_"))
    for idx, url in tqdm(enumerate(image_urls), total=len(image_urls), desc=f"Downloading {category}"):
        file_path = os.path.join(save_path, f"{idx:04d}.jpg")
        try:
            response = requests.get(url, stream=True, timeout=5)
            if response.status_code == 200:
                with open(file_path, "wb") as file:
                    for chunk in response.iter_content(1024):
                        file.write(chunk)
        except Exception as e:
            print(f"Ошибка при загрузке {url}: {e}")

In [9]:

def init_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  # Запуск без интерфейса
    service = Service(ChromeDriverManager().install())  # Автоустановка ChromeDriver
    driver = webdriver.Chrome(service=service, options=options)
    return driver


In [10]:
# Основная логика
def main():
    create_folders()
    driver = init_driver()

    for category in CATEGORIES:
        print(f"🔍 Поиск изображений для: {category}")
        image_urls = fetch_image_urls(driver, category, IMAGES_PER_CATEGORY)
        print(f"✅ Найдено {len(image_urls)} изображений.")
        
        print(f"💾 Начинаем загрузку изображений для: {category}")
        download_images(image_urls, category)
        print(f"🎉 Загрузка {category} завершена!\n")

    driver.quit()

In [11]:

if __name__ == "__main__":
    main()


🔍 Поиск изображений для: polar bear
✅ Найдено 0 изображений.
💾 Начинаем загрузку изображений для: polar bear


🎉 Загрузка polar bear завершена!

🔍 Поиск изображений для: brown bear


✅ Найдено 0 изображений.
💾 Начинаем загрузку изображений для: brown bear


🎉 Загрузка brown bear завершена!

